In [169]:
import ee
import geemap
from geemap import chart

%matplotlib inline
import matplotlib.pyplot as plt

In [170]:
ee.Authenticate()
ee.Initialize()

Enter verification code:  4/1AX4XfWjNrt1jZgMIU7JA8uSxSh9JOTksTjCbZhjUnjChr8dM3betJCOUENo



Successfully saved authorization token.


In [171]:
ls8 = ee.ImageCollection("LANDSAT/LC08/C02/T1_L2")
modis = ee.ImageCollection("MODIS/006/MOD13Q1")
modis2 = ee.ImageCollection('MODIS/006/MCD43A4')
focus = ee.Geometry.Polygon(
        [[[27.350233348102517, -7.518171474050515],
          [27.350233348102517, -7.57841301205225],
          [27.436407359332986, -7.57841301205225],
          [27.436407359332986, -7.518171474050515]]])
WestMine = ee.Geometry.Polygon(
        [[[27.35316117125328, -7.538673727373273],
          [27.35316117125328, -7.562838275361878],
          [27.372730568225936, -7.562838275361878],
          [27.372730568225936, -7.538673727373273]]])
EastMine = ee.Geometry.Polygon(
        [[[27.37513382750328, -7.523187286499756],
          [27.37513382750328, -7.550926343072451],
          [27.41959412413414, -7.550926343072451],
          [27.41959412413414, -7.523187286499756]]])
TailingPond = ee.Geometry.MultiPolygon(
        [[[[27.40117714611331, -7.528029420400128],
           [27.40143463817874, -7.529901416251066],
           [27.40237877575198, -7.5338155632667805],
           [27.404009558833035, -7.535347176386731],
           [27.403837897456082, -7.531603223645428],
           [27.402807929194363, -7.53075232078353],
           [27.402464606440457, -7.527518874674277]]],
         [[[27.401692130244168, -7.537219140622778],
           [27.40160629955569, -7.538835830496198],
           [27.402207114375027, -7.539516540215821],
           [27.40564034191409, -7.539091096766489],
           [27.40564034191409, -7.538410386377943]]],
         [[[27.38813088146487, -7.548791103460967],
           [27.38890335766116, -7.549046363857512],
           [27.391821601069363, -7.5484507560311345],
           [27.389761664545926, -7.546749014863657]]]])
MinePile = ee.Geometry.MultiPolygon(
        [[[[27.39990282562306, -7.538976580191123],
           [27.39863682296803, -7.539487112415201],
           [27.398057465820813, -7.54067835192985],
           [27.397799973755383, -7.542380116977695],
           [27.398883584767866, -7.542497115405488],
           [27.4003963520818, -7.542784285193142],
           [27.401705270081067, -7.541890860211821],
           [27.4014048626714, -7.540295453871944],
           [27.400782588904605, -7.539561565404619]]],
         [[[27.40360114464233, -7.528471399345152],
           [27.40287158379028, -7.53102411635294],
           [27.40506026634643, -7.531151751808675],
           [27.40561816582153, -7.528769217103983]]],
         [[[27.400167917103268, -7.535108432289119],
           [27.400554155201412, -7.5369803975571035],
           [27.403300737232662, -7.537108031257986],
           [27.403944467396236, -7.5352360665416755],
           [27.402013276905514, -7.533491728501599]]],
         [[[27.357092737193568, -7.547347692672706],
           [27.357650636668666, -7.550155558119731],
           [27.359431623454554, -7.550389546083809],
           [27.360375761027797, -7.547985845511823],
           [27.35890590547972, -7.545039700922603],
           [27.357500432963832, -7.545135422206587]]]]);
mines = ee.FeatureCollection("users/rishiAgarwal/Congo_Active_Mines")

In [172]:
rgbVis = {
    'min': 8097.96,
    'max': 12128.04,
    'bands': ['SR_B4', 'SR_B3', 'SR_B2']
}
ndviVis = {
    'min': 0,
    'max': 0.5,
    'palette': ['white', 'green']
}
bsiVis = {
    'min': -0.11390094234742729, 
    'max': 0.12724605991882074, 
    'palette': ['white', '#7a5230']
}
ironVis = {
    'min':0,
    'max':0.5,
     'palette': ['white', 'gray']
}


In [173]:
def create_median_composites(geometry, start, end):
    images = []
    
    for year in range(start, end+1):
        filtered = ls8 \
            .filter(ee.Filter.bounds(geometry)) \
            .filter(ee.Filter.date(str(year) + '-01-01', str(year) + '-12-31')) \
            .filter(ee.Filter.lt('CLOUD_COVER', 15)) \
            .select('SR_B.*') \

        composite = filtered.median().clip(geometry)
        
        images.append((composite, year))
        
    return images

In [174]:
def add_median_composites(images, geometry):
    Map = geemap.Map()
    Map.centerObject(geometry, 13)
    
    for image, year in images:
        Map.addLayer(image, rgbVis, str(year) + ' Median Composite')

    return Map

In [175]:
images = create_median_composites(focus, 2014, 2020)
Map = add_median_composites(images, focus)
Map

Map(center=[-7.548293653363665, 27.393320353717858], controls=(WidgetControl(options=['position', 'transparent…

In [176]:
def extract_data(image):
    stats = image.reduceRegion(**{
        'reducer': ee.Reducer.mean(),
        'geometry': image.geometry(),
        'scale': 100
    })
    
    properties = {
        #'Year': image.get('system:index'),
        'NDVI': stats.get('ndvi'),
        'NDMI': stats.get('ndmi'),
        'GNIR': stats.get('gNIR'),
        'NIRSWIR': stats.get('nirswir'),
        'BSI': stats.get('bsi'),
    }
    return ee.Feature(None, properties)

In [177]:
def add_all_bands(image):
    ndvi = image.normalizedDifference(['SR_B5', 'SR_B4']).rename('ndvi')
    ndmi = image.normalizedDifference(['SR_B5', 'SR_B6']).rename('ndmi')
    gnir = image.normalizedDifference(['SR_B5', 'SR_B3']).rename('gNIR')
    nirswir = image.normalizedDifference(['SR_B5', 'SR_B7']).rename('nirswir')
    bsi = image.expression(
            '(( X + Y ) - (A + B)) /(( X + Y ) + (A + B)) ', {
            'X': image.select('SR_B6'), #swir1
            'Y': image.select('SR_B4'),  #red
            'A': image.select('SR_B5'), # nir
            'B': image.select('SR_B2'), # blue
            }).rename('bsi')
    
    return image.addBands([ndvi, ndmi, gnir, nirswir, bsi])

In [178]:
images = ee.ImageCollection(create_median_composites(focus, 2014, 2020))
images = images.map(add_all_bands)
data = ee.FeatureCollection(images.map(extract_data))


In [179]:
def getData(geometry, dataWanted, year):
    images1 = ee.ImageCollection(create_median_composites(geometry, 2014, 2020))
    with_bands1 = images1.map(add_all_bands)
    data1 = ee.FeatureCollection(with_bands1.map(extract_data))
    df1 = geemap.ee_to_pandas(data1)
    return df1.get(dataWanted).get(year - 2014)

In [180]:
print("WEST MINE")
ndviWest2014 = getData(WestMine, 'NDVI', 2014)
ndviWest2020 = getData(WestMine, 'NDVI', 2020)
bsiWest2014 = getData(WestMine, 'BSI', 2014)
bsiWest2020 = getData(WestMine, 'BSI', 2020)
print('NDVI of West Mine in 2014: ' + str(ndviWest2014))
print('NDVI of West Mine in 2020: ' + str(ndviWest2020))
print("Difference: " + str(ndviWest2020 - ndviWest2014))
print('BSI of West Mine in 2014:' + str(bsiWest2014))
print('BSI of West Mine in 2020:' + str(bsiWest2020))
print('Difference: ' + str(bsiWest2020 - bsiWest2014))
print('Area of West Mine in SqKm: 6 sqKm')
print('Change in NDVI per sqKm for WestMine: ' + str((ndviWest2020 - ndviWest2014) / 6))
print('Change in BSI per sqKm for WestMine: ' + str((bsiWest2020 - bsiWest2014) / 6))
print('\n')
print("EAST MINE")
ndviEast2014 = getData(EastMine, 'NDVI', 2014)
ndviEast2020 = getData(EastMine, 'NDVI', 2020)
bsiEast2014 = getData(EastMine, 'BSI', 2014)
bsiEast2020 = getData(EastMine, 'BSI', 2020)
print('NDVI of East Mine in 2014: ' + str(ndviEast2014))
print('NDVI of East Mine in 2020: ' + str(ndviEast2020))
print("NDVI Difference: " + str(ndviEast2020 - ndviEast2014))
print('BSI of East Mine in 2014: ' + str(bsiEast2014))
print('BSI of East Mine in 2020: ' + str(bsiEast2020))
print('BSI Difference: ' + str(bsiEast2020 - bsiEast2014))
print('Area of West Mine in SqKm: 15.11 sqKm')
print('Change in NDVI per sqKm for EastMine: ' + str((ndviEast2020 - ndviEast2014) / 15.11))
print('Change in NDVI per sqKm for EastMine: ' + str((bsiEast2020 - bsiEast2014)/ 15.11))


WEST MINE
NDVI of West Mine in 2014: 0.25237624123473407
NDVI of West Mine in 2020: 0.2162352379168892
Difference: -0.036141003317844866
BSI of West Mine in 2014:-0.01730392890165352
BSI of West Mine in 2020:0.014418291580219709
Difference: 0.03172222048187323
Area of West Mine in SqKm: 6 sqKm
Change in NDVI per sqKm for WestMine: -0.006023500552974144
Change in BSI per sqKm for WestMine: 0.005287036746978872


EAST MINE
NDVI of East Mine in 2014: 0.24113650655814017
NDVI of East Mine in 2020: 0.20363192232754257
NDVI Difference: -0.037504584230597604
BSI of East Mine in 2014: -0.0005478243749621002
BSI of East Mine in 2020: 0.029133669015066168
BSI Difference: 0.02968149339002827
Area of West Mine in SqKm: 15.11 sqKm
Change in NDVI per sqKm for EastMine: -0.0024821035228721118
Change in NDVI per sqKm for EastMine: 0.0019643609126425063


<h2> Making a Routine </h2>

<h4> Seperating focus region into smaller squares </h4>

In [181]:
def divide_geometry():
    # regionCoordinates = geometry.coordinates()
    regions = []
    i = 27.35
    while i <= 27.44:
        j = -7.52
        while j >= -7.58:
            region = ee.Geometry.Polygon(
                [[[i, j],
                  [i, j-.01],
                  [i+.01, j-.01],
                  [i+.01, j]]])
            regions.append((region))
            j = j-.01
        i = i+.01
    return regions

In [182]:
regions = divide_geometry()
print(len(regions))

63


<h4> Evalutaing Regions by NDVI </h4>

In [183]:
def create_ndvi_layers_modis(start, end, geometry):
    images = []
    ndviStart = modis \
                .filter(ee.Filter.bounds(geometry))\
                .filter(ee.Filter.date(str(start) + '-01-01', str(start) + '-12-31')) \
                .select('NDVI') \
                .median() \
                .clip(geometry)
    images.append((ndviStart, start))
    ndviEnd = modis \
                .filter(ee.Filter.bounds(geometry))\
                .filter(ee.Filter.date(str(end) + '-01-01', str(end) + '-12-31')) \
                .select('NDVI') \
                .median() \
                .clip(geometry)
    images.append((ndviEnd, end))
    return images

In [184]:
def extract_data_modis(image):
    stats = image.reduceRegion(**{
        'reducer': ee.Reducer.mean(),
        'geometry': image.geometry(),
        'scale': 100
    })
    
    properties = {
        'Year': image.get('system:index'),
        'NDVI': stats.get('NDVI')
    }
    
    return ee.Feature(None, properties)

In [185]:
def ndvi_requirement_modis(df):
    statusM = 0 # 1 = possible mine, 0 = not mine
    if df.iat[0,1]-df.iat[1,1] > 500:
        statusM = 1
    else: 
        statusM = statusM
    return statusM

In [225]:
def ndvi_level_modis(regions):
    Map = geemap.Map()
    Map.setCenter(27.3954, -7.55, 13)
    #Map.addLayer(comp2020,rgbVis,'whole',False)
    num = 1
    for region in regions:
        layerName = str(num) + ' ndvi pass'
        
        images = ee.ImageCollection(create_ndvi_layers_modis(2004, 2020, region))
        data = ee.FeatureCollection(images.map(extract_data_modis))
        df = geemap.ee_to_pandas(data)

        statusM = ndvi_requirement_modis(df)
        if statusM == 1:
            image = ls8 \
                .filter(ee.Filter.bounds(region)) \
                .filter(ee.Filter.date('2020-01-01', '2020-12-31')) \
                .filter(ee.Filter.lt('CLOUD_COVER', 15)) \
                .select('SR_B.*') \
                .median().clip(region)
            Map.addLayer(image, rgbVis, layerName)
        num = num+1
    return Map

In [226]:
ndvi_level_modis(regions)

Map(center=[-7.55, 27.3954], controls=(WidgetControl(options=['position', 'transparent_bg'], widget=HBox(child…

<h4> Evalutating Regions by BSI </h4>

In [67]:
def create_bsi_layers_modis(start, end, geometry):
    images = []
    bsiStart = modis2 \
                .filter(ee.Filter.bounds(geometry))\
                .filter(ee.Filter.date(str(start) + '-01-01', str(start) + '-12-31')) \
                .median() \
                .clip(geometry)
    startBsi = bsiStart.expression(
        '(( X + Y ) - (A + B)) /(( X + Y ) + (A + B)) ', {
            'X': bsiStart.select('Nadir_Reflectance_Band2'), #swir1
            'Y': bsiStart.select('Nadir_Reflectance_Band1'),  #red
            'A': bsiStart.select('Nadir_Reflectance_Band2'), # nir
            'B': bsiStart.select('Nadir_Reflectance_Band3'), # blue
            }).rename('bsi')
    images.append((startBsi.select('bsi'), start))
    bsiEnd = modis2 \
                .filter(ee.Filter.bounds(geometry))\
                .filter(ee.Filter.date(str(end) + '-01-01', str(end) + '-12-31')) \
                .median() \
                .clip(geometry)
    endBsi = bsiEnd.expression(
        '(( X + Y ) - (A + B)) /(( X + Y ) + (A + B)) ', {
            'X': bsiStart.select('Nadir_Reflectance_Band2'), #swir1
            'Y': bsiStart.select('Nadir_Reflectance_Band1'),  #red
            'A': bsiStart.select('Nadir_Reflectance_Band2'), # nir
            'B': bsiStart.select('Nadir_Reflectance_Band3'), # blue
            }).rename('bsi')
    images.append((endBsi.select('bsi'), end))
    return images

In [68]:
def extract_data_modis_bsi(image):
    stats = image.reduceRegion(**{
        'reducer': ee.Reducer.mean(),
        'geometry': image.geometry(),
        'scale': 100
    })
    
    properties = {
        'Year': image.get('system:index'),
        'BSI': stats.get('bsi')
    }
    
    return ee.Feature(None, properties)

In [69]:
def bsi_requirement_modis(df):
    statusM = 0 # 1 = possible mine, 0 = not mine
    if df.iat[0,1]-df.iat[1,1] > 500:
        statusM = 1
    else: 
        statusM = statusM
    return statusM

In [72]:
def bsi_level_modis(regions):
    Map = geemap.Map()
    Map.setCenter(27.3954, -7.55, 13)
    #Map.addLayer(comp2020,rgbVis,'whole',False)
    num = 1
    for region in regions:
        layerName = str(num) + ' bsi pass'
        
        images = ee.ImageCollection(create_bsi_layers_modis(2014, 2020, region))
        images = images.map(add_all_bands)
        data = ee.FeatureCollection(images.map(extract_data_modis_bsi))
        df = geemap.ee_to_pandas(data)

        statusM = bsi_requirement_modis(df)
        if statusM == 1:
            image = ls8 \
                .filter(ee.Filter.bounds(region)) \
                .filter(ee.Filter.date('2020-01-01', '2020-12-31')) \
                .filter(ee.Filter.lt('CLOUD_COVER', 15)) \
                .select('SR_B.*') \
                .median().clip(region)
            Map.addLayer(image, bsiVis, layerName)
        num = num+1
    return Map

In [73]:
bsi_level_modis(regions)

Exception: Error in map(ID=1):
Image.normalizedDifference: No band named 'SR_B5'. Available band names: [bsi, constant].

<h3> Evaluating Regions by Iron </h3>

In [188]:
def create_iron_layers(start, end, geometry):
    images = []
    for year in range(start, end):
        image = modis2 \
        .filter(ee.Filter.bounds(geometry))\
        .filter(ee.Filter.date(str(year) + '-01-01', str(year) + '-12-31')) \
        .median() \
        .clip(geometry)
        iron = image.normalizedDifference(['Nadir_Reflectance_Band1', 'Nadir_Reflectance_Band3']).rename('IRON')
        images.append((iron, year))

    return images

In [189]:
def extract_data_modis_iron(image):
    stats = image.reduceRegion(**{
        'reducer': ee.Reducer.mean(),
        'geometry': image.geometry(),
        'scale': 100
    })
    
    properties = {
        'Year': image.get('system:index'),
        'IRON': stats.get('IRON')
    }
    
    return ee.Feature(None, properties)

In [190]:
def iron_requirement_modis(df):
    #print(df)
    statusM = 0 # 1 = possible mine, 0 = not mine
    if (df.iat[15,1]-df.iat[0,1] > 0.020) or (df.iat[0,1] < 0.03):
        statusM = 1
    else: 
        statusM = statusM
    return statusM

In [192]:
def iron_level_modis(regions):
    Map = geemap.Map()
    Map.setCenter(27.3954, -7.55, 13)
    #Map.addLayer(comp2020,rgbVis,'whole',False)
    num = 1
    for region in regions:
        layerName = str(num) + ' ndvi pass'
        
        images = ee.ImageCollection(create_iron_layers(2004, 2020, region))
        data = ee.FeatureCollection(images.map(extract_data_modis_iron))
        df = geemap.ee_to_pandas(data)

        statusM = iron_requirement_modis(df)
        if statusM == 1:
            image = ls8 \
                .filter(ee.Filter.bounds(region)) \
                .filter(ee.Filter.date('2020-01-01', '2020-12-31')) \
                .filter(ee.Filter.lt('CLOUD_COVER', 15)) \
                .select('SR_B.*') \
                .median().clip(region)
            Map.addLayer(image, rgbVis, layerName)
        num = num+1
    return Map

In [196]:
iron_level_modis(regions)

Map(center=[-7.55, 27.3954], controls=(WidgetControl(options=['position', 'transparent_bg'], widget=HBox(child…

<h4> Evaluating Regions by NDMI </h4>

In [197]:
def create_ndmi_layers(start, end, geometry):
    images = []
    for year in range(start, end):
        image = modis2 \
        .filter(ee.Filter.bounds(geometry))\
        .filter(ee.Filter.date(str(year) + '-01-01', str(year) + '-12-31')) \
        .median() \
        .clip(geometry)
        ndmi = image.normalizedDifference(['Nadir_Reflectance_Band2', 'Nadir_Reflectance_Band6']).rename('NDMI')
        images.append((ndmi, year))

    return images

In [198]:
def extract_data_modis_ndmi(image):
    stats = image.reduceRegion(**{
        'reducer': ee.Reducer.mean(),
        'geometry': image.geometry(),
        'scale': 100
    })
    
    properties = {
        'Year': image.get('system:index'),
        'NDMI': stats.get('NDMI')
    }
    
    return ee.Feature(None, properties)

In [227]:
def ndmi_requirement_modis(df):
    #print(df)
    statusM = 0 # 1 = possible mine, 0 = not mine
    if (df.iat[0,1]-df.iat[15,1] > 0.1b 0):
        statusM = 1
    else: 
        statusM = statusM
    return statusM

In [228]:
def ndmi_level_modis(regions):
    Map = geemap.Map()
    Map.setCenter(27.3954, -7.55, 13)
    #Map.addLayer(comp2020,rgbVis,'whole',False)
    num = 1
    for region in regions:
        layerName = str(num) + ' nmdi pass'
        
        images = ee.ImageCollection(create_ndmi_layers(2004, 2020, region))
        data = ee.FeatureCollection(images.map(extract_data_modis_ndmi))
        df = geemap.ee_to_pandas(data)
        #print(num)
        statusM = ndmi_requirement_modis(df)
        if statusM == 1:
            image = ls8 \
                .filter(ee.Filter.bounds(region)) \
                .filter(ee.Filter.date('2020-01-01', '2020-12-31')) \
                .filter(ee.Filter.lt('CLOUD_COVER', 15)) \
                .select('SR_B.*') \
                .median().clip(region)
            Map.addLayer(image, rgbVis, layerName)
        num = num+1
    return Map

In [229]:
ndmi_level_modis(regions)

Map(center=[-7.55, 27.3954], controls=(WidgetControl(options=['position', 'transparent_bg'], widget=HBox(child…